In [4]:
import json
import geopandas as gp
from shapely.geometry import Point, LineString, Polygon

In [2]:
# these are the unedited data from Ollie O'Brien's vis repo (https://github.com/oobrien/vis)
with open("static/tfl_lines.geojson", 'r') as f:            
    gj = json.loads(f.read()) 

In [3]:
# Some segments belong to multiple lines
# We need to build new LineStrings with each of these
# this results in a small amount of duplication but that's OK
new_features = []
for feature in gj['features']:
    num_lines = len(feature['properties']['lines'])
    if num_lines > 1:
        for segment in feature['properties']['lines'][1:]:
            new_feature = feature
            new_feature['properties']['line_name'] = segment['name']
            new_feature['properties']['shared'] = True
            new_features.append(
                new_feature
            )
    else:
        feature['properties']['shared'] = False

In [4]:
# assign a new line_name property to existing features
for feature in gj['features']:                              
    feature['properties']['line_name'] = feature['properties']['lines'][0]['name']

In [ ]:
# combine existing and new features
gj['features'] = gj['features'] + new_features

In [6]:
# remove 'lines' property, since we don't need it any more
for feature in gj['features']:
    try:
        del feature['properties']['lines']
    except KeyError:
        pass

In [7]:
with open("static/tfl_lines_edited.geojson", 'w') as g:
    json.dump(gj, g)

In [5]:
df = gp.read_file("static/tfl_lines_edited.geojson")

In [6]:
# Mapbox GL can only extrude polygons.
# LineStrings can be converted into Polygons by buffering them
# choose an appropriate width here
df.geometry = df.geometry.buffer(0.000100)

In [7]:
# fix up other unfamiliar lines
df['line_name'] = df['line_name'].replace('East London', 'London Overground')
df['line_name'] = df['line_name'].replace('Crossrail 2', 'Crossrail')
df['line_name'] = df['line_name'].replace('TfL Rail', 'Crossrail')

In [10]:
df.line_name.unique()

array([u'London Overground', u'Victoria', u'Piccadilly', u'District',
       u'Circle', u'Hammersmith & City', u'Metropolitan', u'Central',
       u'Jubilee', u'DLR', u'Bakerloo', u'Northern', u'Waterloo & City',
       'Crossrail', u'Tramlink', u'Emirates Air Line'], dtype=object)


In [36]:
df.to_file("static/tube_polygons.geojson", driver="GeoJSON")